# FunOS module init analysis using raw log

In [ ]:

# This is a notebook that generates a chart of module init time. 
# This notebook shows the module init time chart.

# Input:
#     - raw log file: load through config file

# Config file:
#    - config file: `funos_module_init_raw_input_config.yml` in the same directory as this notebook

In [ ]:
# Generate html file
# $ python ./convert_nb.py --filename ./funos_module_init_raw_input.ipynb --execute

In [ ]:
# *NOTE*: run this command to clean output cell and meta data.
# $ nb-clean clean ./funos_module_init_raw_input.ipynb 

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from typing import Tuple
import yaml
import json
import os
import logging

%matplotlib inline

import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 22})

pd.options.display.float_format = '{:,}'.format

#https://stackoverflow.com/questions/36288670/how-to-programmatically-generate-markdown-output-in-jupyter-notebooks
from IPython.display import display, Markdown, Latex

In [ ]:
try:
    from funos_module_init_time import *
    from funos_module_init_time import process_module_notif_init_raw_data
    from funos_module_init_time_plot import plot_module_time_chart
except ImportError as e:
    print("Import error: {}".format(e))
    print("Set this first:")
    print("$ export PYTHONPATH=$WORKSPACE/FunTools/data_analysis:$PYTHONPATH")
    assert(False)

## Load config

In [ ]:
# load config file: funos_module_init_analysis_config.yml
# - config file can be found from either using env var or from the current directory

current_path = os.getcwd()

if "FUNOS_MODULE_INIT_ANALYSIS_CONFIG_FILE" in os.environ:
    config_file_name = os.environ["FUNOS_MODULE_INIT_ANALYSIS_CONFIG_FILE"]
    print("Using env[FUNOS_MODULE_INIT_ANALYSIS_CONFIG_FILE] config file: {}".format(config_file_name))
else:
    config_file = "configs/funos_module_init_raw_input_config.yml"
    config_file_name = os.path.join(current_path, config_file)

print("current working directory is: "+current_path)
print("config file " + config_file_name)

try:
    with open (config_file_name, 'r') as c_file:
        config = yaml.safe_load(c_file)
except Exception as e:
    print('Error reading the config file at {} : {}'.format(config_file_name, e))

In [ ]:
# setup the config variables
input_file_url = config["file_names"]["input_file_url"]
input_description = config["file_names"]["description"]
out_dir = config["out_dir"]

In [ ]:
def printmd(string):  ###
    display(Markdown(string))  ###

## Notebook summary

In [ ]:

note_str = "This page summarizes the FunOS modules init time line.\n\n"
printmd(note_str)


## Module init loading

In [ ]:
# A logger for this file
logger = logging.getLogger(__name__)
# logger.setLevel(logging.DEBUG)


In [ ]:
# load config file
current_path = os.getcwd()
note_str = "current directory is: {}\n\n".format(current_path)
printmd(note_str)

note_str = "INPUT_FILE_URL: [{}]({})\n\n".format(input_file_url, input_file_url)
printmd(note_str)

fun_module_init_df, result = process_module_notif_init_raw_data(
    file_name_url=input_file_url, logger=logger, working_dir=out_dir
)

note_str = "### Module init time summary\n\n"
printmd(note_str)

note_str = "Description: {}\n\n".format(input_description)
printmd(note_str)

# tmp_df = pd.DataFrame(result, index=[0])
tmp_df = pd.DataFrame(result, index=["µsec"])
pd.options.display.float_format = '{:,.0f}'.format

# convert to µsec
num_cols = ["longest_duration_ns", "longest_gap_ns", "total_module_init_time_only_ns", "total_duration_ns"]
for col in num_cols: tmp_df[col] /= 1000 # convert to µsec

tmp_df.rename(
    columns={
        "longest_duration_ns": "longest_duration (µsec)",
        "longest_gap_ns": "longest_gap (µsec)",
        "total_module_init_time_only_ns": "total_module_init_time_only (µsec)",
        "total_duration_ns": "total_duration (µsec)",
    },
    inplace=True,
)
display(tmp_df.T.round())

top_list = 15
note_str = f"### Top {top_list} module init durations (in µsec)\n\n"
printmd(note_str)

note_str = "Description: {}\n\n".format(input_description)
printmd(note_str)

# sort by module init time
fun_module_init_df.sort_values(by=['module_init_duration'], ascending=False, inplace=True)
# convert to µsec
fun_module_init_df_tmp = fun_module_init_df.copy()
fun_module_init_df_tmp["module_init_duration"] /= 1000
fun_module_init_df_tmp.rename(
    columns={
        "module_init_duration": "module_init_duration (µsec)",
    },
    inplace=True,
)
# display module_init_duration only
display(fun_module_init_df_tmp.head(top_list)[['module_init_duration (µsec)']].round(1))

del tmp_df
del fun_module_init_df_tmp


## Plot all modules and notif init

In [ ]:
# MAIN CALLS
# plot the events

note_str = "The following chart shows the modules and notification init durations and timelines.\n\n"
printmd(note_str)

threshold_collapse = get_duration_threshold(fun_module_init_df, threshold=0.01)

note_str = "**NOTE**: When the duration is less than a threshold ({} ns), the module duration is depicted as the threshold duration so that it can be seen in the chart.\n\n".format(int(threshold_collapse))

printmd(note_str)

save_file_name = os.path.join(out_dir, "fun_module_init_chart.png")
plot_module_time_chart(fun_module_init_df,save_file_name=save_file_name, disp_granualarity_ms=1000, min_duration=threshold_collapse, debug=False,logger=logger)